In [1]:
import pandas as pd
import os

## Vista previa de un conjunto de datos

In [ ]:
root_dir = "../dataset/raw"

In [3]:
pd.read_csv(root_dir + "/A1/humidity1.csv").head()

,Device,Slot,Epoch,Timestamp,Value
0,A1-GeoNodo 01,Humedad 1,1698177537000,2023-10-24 16:58:57,0.0
1,A1-GeoNodo 01,Humedad 1,1698177596000,2023-10-24 16:59:56,0.0
2,A1-GeoNodo 01,Humedad 1,1698177656000,2023-10-24 17:00:56,0.0
3,A1-GeoNodo 01,Humedad 1,1698177716000,2023-10-24 17:01:56,0.0
4,A1-GeoNodo 01,Humedad 1,1698177747000,2023-10-24 17:02:27,0.0


## Columnas originales de cada conjunto de datos.
* Device: Nombre del dispositivo, en este caso, "A1-GeoNodo 01".
* Slot: Variable de cada subconjunto de datos ("Humedad 1", "Humedad 2", etc.).
* Epoch: Unix timestamp.
* Timestamp: Fecha y hora de cada registro.
* Value: Valor de la medición de humedad, pH o temperatura.

---

# Análisis conjunto de los archivos CSV

---
## Concatenación de los conjuntos de datos en un DataFrame
Y creación de una columna adicional ("Node") para identificar el nodo correspondiente a cada registro. 

In [4]:
all_dataframes = []

for node in os.listdir(root_dir):
    node_path = os.path.join(root_dir, node)

    if os.path.isdir(node_path):
        for file in os.listdir(node_path):
            file_path = os.path.join(node_path, file)

            if file.endswith(".csv"):
                df = pd.read_csv(file_path)

                df["Node"] = node

                all_dataframes.append(df)

df_combined = pd.concat(all_dataframes, ignore_index=True)

In [5]:
df_combined

,Device,Slot,Epoch,Timestamp,Value,Node
0,A1-GeoNodo 01,Humedad 1,1698177537000,2023-10-24 16:58:57,0.0,A1
1,A1-GeoNodo 01,Humedad 1,1698177596000,2023-10-24 16:59:56,0.0,A1
2,A1-GeoNodo 01,Humedad 1,1698177656000,2023-10-24 17:00:56,0.0,A1
3,A1-GeoNodo 01,Humedad 1,1698177716000,2023-10-24 17:01:56,0.0,A1
4,A1-GeoNodo 01,Humedad 1,1698177747000,2023-10-24 17:02:27,0.0,A1
...,...,...,...,...,...,...
2057141,D4-GeoNodo 15,Temperatura 2,1705597316000,2024-01-18 14:01:56,22.5,D4
2057142,D4-GeoNodo 15,Temperatura 2,1705597376000,2024-01-18 14:02:56,22.5,D4
2057143,D4-GeoNodo 15,Temperatura 2,1705597436000,2024-01-18 14:03:56,22.5,D4
2057144,D4-GeoNodo 15,Temperatura 2,1705597616000,2024-01-18 14:06:56,22.5,D4


---

## Convertir columnas "Epoch" y "Timestamp" a *datetime*

In [6]:
df_combined["Timestamp"] = pd.to_datetime(df_combined["Timestamp"])
df_combined["Epoch"] = pd.to_datetime(df_combined["Epoch"],
                                      unit="ms",
                                      origin="unix")

In [7]:
df_combined.head()

,Device,Slot,Epoch,Timestamp,Value,Node
0,A1-GeoNodo 01,Humedad 1,2023-10-24 19:58:57,2023-10-24 16:58:57,0.0,A1
1,A1-GeoNodo 01,Humedad 1,2023-10-24 19:59:56,2023-10-24 16:59:56,0.0,A1
2,A1-GeoNodo 01,Humedad 1,2023-10-24 20:00:56,2023-10-24 17:00:56,0.0,A1
3,A1-GeoNodo 01,Humedad 1,2023-10-24 20:01:56,2023-10-24 17:01:56,0.0,A1
4,A1-GeoNodo 01,Humedad 1,2023-10-24 20:02:27,2023-10-24 17:02:27,0.0,A1


---

## Consultar información general del DataFrame resultante

In [8]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2057146 entries, 0 to 2057145
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Device     object        
 1   Slot       object        
 2   Epoch      datetime64[ns]
 3   Timestamp  datetime64[ns]
 4   Value      float64       
 5   Node       object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 94.2+ MB


---

## Tipos de datos (variables)
* **Device:** Categórica nominal.
* **Slot:** Categórica nominal.
* **Epoch:** Temporal.
* **Timestamp:** Temporal.
* **Value:** Cuantitativa continua
    * *Temperature*: Escala de intervalo.
    * *Humidity*: Escala de razón.
    * *pH*: Escala de intervalo.

---

# Verificar posibles nulos y duplicados

In [9]:
# Valores nulos en todas las filas y columnas
df_combined.isnull().sum()

Device       0
Slot         0
Epoch        0
Timestamp    0
Value        0
Node         0
dtype: int64

In [10]:
# Duplicados a nivel de filas
df_combined.duplicated().sum()

np.int64(0)

---

# Verificar valores únicos en columnas clave

In [11]:
print(df_combined["Device"].unique())
print(df_combined["Slot"].unique())

['A1-GeoNodo 01' 'A2-GeoNodo 07' 'A3-GeoNodo 03' 'A4-GeoNodo 04'
 'B1-GeoNodo 10' 'B2-GeoNodo 11' 'B3-GeoNodo 09' 'B4-GeoNodo 13'
 'C1-GeoNodo 02' 'C2-GeoNodo 16' 'C3-GeoNodo 14' 'C4-GeoNodo 12'
 'D1-GeoNodo 05' 'D2-GeoNodo 06' 'D3-GeoNodo 08' 'D4-GeoNodo 15']
['Humedad 1' 'Humedad 2' 'pH' 'Temperatura 1' 'Temperatura 2']


## Resultados iniciales
* Las columnas están con sus tipos de datos correspondientes, a excepción de "Epoch" y "Timestamp", los que se pueden convertir a *datetime*.
* La conversión de ambas columnas a *datetime* resulta en dos timestamps con una diferencia de 3 horas entre el valor de una y la otra.
* "Epoch" posee un timestamp con 3 horas por sobre "Timestamp".
* No hay duplicados a nivel de filas.
* No hay valores nulos.

---

## (EXTRA)

---

## Crear tabla pivote para fusionar filas con un mismo "Timestamp" y "Node"
Crear una tabla pivote en la que cada fila representa un "Timestamp" y "Node" únicos, con columnas adicionales para cada tipo de medición ("Slot").

In [12]:
pivot_df = df_combined.pivot_table(index=["Timestamp", "Node"],  
                                   columns="Slot",               
                                   values="Value",             
                                  )

In [13]:
pivot_df

,Slot,Humedad 1,Humedad 2,Temperatura 1,Temperatura 2,pH
Timestamp,Node,,,,,
2023-10-24 16:56:58,A3,0.0,0.0,38.3,36.9,655.35
2023-10-24 16:57:28,A3,0.0,0.0,38.3,36.9,655.35
2023-10-24 16:57:58,A3,0.0,0.0,38.3,37.0,655.35
2023-10-24 16:58:28,A3,0.0,0.0,38.5,37.1,655.35
2023-10-24 16:58:57,A1,0.0,0.0,50.7,49.3,7.84
...,...,...,...,...,...,...
2024-01-18 14:12:43,D3,32.4,28.5,23.3,22.7,655.35
2024-01-18 14:12:49,B1,25.4,28.8,23.6,23.0,4.67
2024-01-18 14:12:55,A4,24.1,32.8,23.8,23.2,655.35


In [14]:
# Ordenar primero por "Node" y luego por "Timestamp"
pivot_df_sorted = pivot_df.sort_values(by=["Node", 
                                           "Timestamp"])

In [15]:
pivot_df_sorted

,Slot,Humedad 1,Humedad 2,Temperatura 1,Temperatura 2,pH
Timestamp,Node,,,,,
2023-10-24 16:58:57,A1,0.0,0.0,50.7,49.3,7.84
2023-10-24 16:59:56,A1,0.0,0.0,50.8,49.4,7.84
2023-10-24 17:00:56,A1,0.0,0.0,50.8,49.4,7.83
2023-10-24 17:01:56,A1,0.0,0.0,50.6,49.4,7.83
2023-10-24 17:02:27,A1,0.0,0.0,50.5,49.4,7.83
...,...,...,...,...,...,...
2024-01-18 14:01:56,D4,24.5,18.8,23.2,22.5,655.35
2024-01-18 14:02:56,D4,24.5,18.8,23.2,22.5,655.35
2024-01-18 14:03:56,D4,24.5,18.8,23.2,22.5,655.35


---

# Comparación entre CSV individuales

---

Primero, se crea una lista que contendrá las rutas de todas las subcarpetas, para así poder extraer la data de los archivos de cada nodo

In [16]:
subfolders = [folder.path 
              for folder in os.scandir(root_dir) 
              if folder.is_dir()]

## Obtener las columnas de cada archivo CSV

Se crea una función para extraer y almacenar solo los nombres de las columnas de todos los DataFrames cargados desde los archivos CSV en el directorio especificado

In [17]:
def get_csv_columns(dir_path):
    columns_set = set()
    for filename in os.listdir(dir_path):
        if filename.endswith(".csv"):
            df = pd.read_csv(os.path.join(dir_path, filename))
            columns_set.update(df.columns)
    return columns_set

## Comparar los archivos CSV en cuanto a cantidad y nombres de sus columnas
Esto, con el fin de comprobar si todos los conjuntos de datos cuentan con las mismas columnas

In [18]:
comparison_results = []
for i in range(len(subfolders)):
    for j in range(i + 1, len(subfolders)):
        cols_a = get_csv_columns(subfolders[i])
        cols_b = get_csv_columns(subfolders[j])

        if cols_a.difference(cols_b) == set() and cols_b.difference(cols_a) == set():
            result = f"Las columnas son iguales"
        else:
            result = f"Las columnas son diferentes"

        comparison_results.append({"Subfolder 1": subfolders[i],
                                   "Subfolder 2": subfolders[j],
                                   "Result": result})

In [19]:
comparison_results_df = pd.DataFrame(comparison_results)
comparison_results_df

,Subfolder 1,Subfolder 2,Result
0,../data/raw\A1,../data/raw\A2,Las columnas son iguales
1,../data/raw\A1,../data/raw\A3,Las columnas son iguales
2,../data/raw\A1,../data/raw\A4,Las columnas son iguales
3,../data/raw\A1,../data/raw\B1,Las columnas son iguales
4,../data/raw\A1,../data/raw\B2,Las columnas son iguales
...,...,...,...
115,../data/raw\D1,../data/raw\D3,Las columnas son iguales
116,../data/raw\D1,../data/raw\D4,Las columnas son iguales
117,../data/raw\D2,../data/raw\D3,Las columnas son iguales
118,../data/raw\D2,../data/raw\D4,Las columnas son iguales


Verificamos el resultado de las comparaciones entre todos los archivos CSV

In [20]:
comparison_results_df["Result"].value_counts()

Result
Las columnas son iguales    120
Name: count, dtype: int64

Como se observa, el resultado de todas las comparaciones indica que todos los conjuntos de datos comparten las mismas columnas.

---

---

## Cantidad de valores NaN por columnas de cada CSV

En la primera parte del notebook se juntaron todos los archivos en un solo DataFrame y se realizó la búsqueda de valores **NaN** (nulos) a lo largo y ancho de éste. A pesar de no haber obtenido valores nulos, en las siguientes celdas se procede a realizar una búsqueda archivo por archivo, para cerciorarnos aún más.

Función para extraer un conjunto de datos (CSV) y almacenarlo en un DataFrame

In [21]:
def get_csv(dir_path):
    csv_list = list()
    csv_names = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".csv"):
            df = pd.read_csv(os.path.join(dir_path, filename))
            csv_list.append(df)
            csv_names.append(os.path.splitext(filename)[0])
    return csv_list, csv_names

Se crea una lista para almacenar todos los DataFrames generados a partir de los archivos CSV

In [22]:
csv_superlist = []
csv_superlist_names = []
for i in range(len(subfolders)):
    csvs, names = get_csv(subfolders[i])
    csv_superlist.append(csvs)
    csv_superlist_names.append(names)

Para comprobar los resultados, se itera por todos los DataFrames contenidos en la lista

In [23]:
for index, df_list in enumerate(csv_superlist):
    print(f"Subcarpeta {subfolders[index]} tiene {len(df_list)} DataFrames")

Subcarpeta ../data/raw\A1 tiene 5 DataFrames
Subcarpeta ../data/raw\A2 tiene 5 DataFrames
Subcarpeta ../data/raw\A3 tiene 5 DataFrames
Subcarpeta ../data/raw\A4 tiene 5 DataFrames
Subcarpeta ../data/raw\B1 tiene 5 DataFrames
Subcarpeta ../data/raw\B2 tiene 5 DataFrames
Subcarpeta ../data/raw\B3 tiene 5 DataFrames
Subcarpeta ../data/raw\B4 tiene 5 DataFrames
Subcarpeta ../data/raw\C1 tiene 5 DataFrames
Subcarpeta ../data/raw\C2 tiene 5 DataFrames
Subcarpeta ../data/raw\C3 tiene 5 DataFrames
Subcarpeta ../data/raw\C4 tiene 5 DataFrames
Subcarpeta ../data/raw\D1 tiene 5 DataFrames
Subcarpeta ../data/raw\D2 tiene 5 DataFrames
Subcarpeta ../data/raw\D3 tiene 5 DataFrames
Subcarpeta ../data/raw\D4 tiene 5 DataFrames


Efectivamente, la lista "csv_superlist" contiene, a su vez, una lista de 5 DataFrames.
Cada Dataframe corresponde a cada archivo ("humidity1", "humidity2", "ph", "temperature1" y "temperature") de cada nodo (A1, A2..., B1, B2..., etc.).

A continuación, para poder contar el número de valores **NaN** de las columnas de cada DataFrame  

In [24]:
# Iterar sobre cada sublista y, dentro de ésta, por cada DataFrame
nan_results = []

for folder_index, df_list in enumerate(csv_superlist):
    for df_index, df in enumerate(df_list):
        nan_count = df.isna().sum()
        for column, count in nan_count.items():
            nan_results.append({
                "Subcarpeta": subfolders[folder_index],
                "DataFrame": csv_superlist_names[folder_index][df_index],
                "Columna": column,
                "NaN Count": count
            })

In [25]:
nan_results_df = pd.DataFrame(nan_results)
nan_results_df

,Subcarpeta,DataFrame,Columna,NaN Count
0,../data/raw\A1,humidity1,Device,0
1,../data/raw\A1,humidity1,Slot,0
2,../data/raw\A1,humidity1,Epoch,0
3,../data/raw\A1,humidity1,Timestamp,0
4,../data/raw\A1,humidity1,Value,0
...,...,...,...,...
395,../data/raw\D4,temperature2,Device,0
396,../data/raw\D4,temperature2,Slot,0
397,../data/raw\D4,temperature2,Epoch,0
398,../data/raw\D4,temperature2,Timestamp,0


Verificamos el resultado del recuento de valores **NaN**

In [26]:
nan_results_df["NaN Count"].value_counts()

NaN Count
0    400
Name: count, dtype: int64

Nuevamente, no obtenemos ningún valor nulo a lo largo y ancho de cada conjunto de datos.

---

---

## Cantidad de filas (registros) por CSV

In [27]:
row_results = []

for folder_index, df_list in enumerate(csv_superlist):
    for df_index, df in enumerate(df_list):
        row_count = len(df)
        row_results.append({
            "Subcarpeta": subfolders[folder_index],
            "DataFrame": csv_superlist_names[folder_index][df_index],
            "Row Count": row_count
        })

In [28]:
row_results_df = pd.DataFrame(row_results)
row_results_df

,Subcarpeta,DataFrame,Row Count
0,../data/raw\A1,humidity1,19673
1,../data/raw\A1,humidity2,19709
2,../data/raw\A1,ph,19745
3,../data/raw\A1,temperature1,19703
4,../data/raw\A1,temperature2,19670
...,...,...,...
75,../data/raw\D4,humidity1,45497
76,../data/raw\D4,humidity2,45440
77,../data/raw\D4,ph,45498
78,../data/raw\D4,temperature1,45154


Para poder visualizar el recuento de filas de cada DataFrame (y, por lo tanto, su CSV correspondiente), se necesita pivotar la tabla anterior.

In [29]:
row_count_pivot = row_results_df.pivot(index="Subcarpeta",
                                       columns="DataFrame",
                                       values="Row Count")
row_count_pivot

DataFrame,humidity1,humidity2,ph,temperature1,temperature2
Subcarpeta,,,,,
../data/raw\A1,19673,19709,19745,19703,19670
../data/raw\A2,40477,40372,40528,40482,40222
../data/raw\A3,42193,41976,42198,42135,41867
../data/raw\A4,27488,27237,27493,27488,27269
../data/raw\B1,28604,28454,28604,28604,28403
../data/raw\B2,18816,18734,18836,18730,18671
../data/raw\B3,14147,14141,14147,14092,14118
../data/raw\B4,44376,44272,44424,44114,44016
../data/raw\C1,17947,17796,17969,17938,17709


---

## Descargar CSV con los datasets concatenados (descomentar para ejecutar)

Por último, se puede obtener un archivo CSV en que se concatenen los archivos CSV de todas las subcarpetas de los nodos, en que se incluye una columna que indica el conjunto de datos al que pertenece cada registro. Asimismo, se puede descargar para su uso posterior.

Es importante destacar que, para este caso, las columnas "Epoch" y "Timestamp" no están convertidas a *datetime*.

In [ ]:
df_combined.to_csv("../dataset/processed/all_nodes.csv", index=False)

___

# Resumen de resultados

Para terminar, se desglosan los resultados de todos los requerimientos cubiertos en el presente documento.

## Información general de los conjuntos de datos
* Son 80 archivos CSV en total.
* La data comprende los registros de 16 nodos.
* Cada dispositivo cuenta con 5 archivos.
* Cada archivo cuenta con 5 columnas.

### Columnas originales de cada CSV
* **"Device"**: Nombre del dispositivo.
* **"Slot"**: Variable de cada subconjunto de datos ("Humedad 1", "Humedad 2", etc.).
* **"Epoch"**: Unix Timestamp de cada registro de medición (equivalente a UTC 0)
* **"Timestamp"**: Fecha y hora de cada registro de medición, en UTC-3.
* **"Value"**: Valor de la medición de humedad, pH o temperatura.

### Tipos de datos (variables)
* **Device:** Categórica nominal.
* **Slot:** Categórica nominal.
* **Epoch:** Temporal.
* **Timestamp:** Temporal.
* **Value:** Cuantitativa continua
    * *Temperature*: Escala de intervalo.
    * *Humidity*: Escala de razón.
    * *pH*: Escala de intervalo.

### Dispositivos y las mediciones
A través de la verificación de los valores únicos en las columnas "Device" y "Slot", se extrajeron los siguientes dispositivos y mediciones:
* Dispositivos:
    * A1-GeoNodo 01 | A2-GeoNodo 07 | A3-GeoNodo 03 | A4-GeoNodo 04
    * B1-GeoNodo 10 | B2-GeoNodo 11 | B3-GeoNodo 09 | B4-GeoNodo 13
    * C1-GeoNodo 02 | C2-GeoNodo 16 | C3-GeoNodo 14 | C4-GeoNodo 12
    * D1-GeoNodo 05 | D2-GeoNodo 06 | D3-GeoNodo 08 | D4-GeoNodo 15

* Mediciones por dispositivo:
    * "Humedad 1" (sensor a 1 metro de profundidad)
    * "Humedad 2" (sensor a 1.5 metro de profundidad)
    * "pH"
    * "Temperatura 1" (sensor a 1 metro de profundidad)
    * "Temperatura 2" (sensor a 1.5 metro de profundidad)


## Dimensiones de los conjuntos de datos

### Cantidad y nombres de columnas
Mediante código, se realizó una comparación entre los archivos CSV de cada dispositivo, con el fin de comprobar si todos los conjuntos de datos cuentan con las mismas columnas.

Con un total de 120 comparaciones, se obtuvo que todos los archivos poseen la misma cantidad y los mismos nombres de columnas.

---

### Cantidad de filas por archivo CSV
Con un total de 80 archivos CSV, cada uno cuenta con respectiva cantidad de registros:

| Dispositivo | humidity1 | humidity2 |   ph   | temperature1 | temperature2 |
|-------------|-----------|-----------|--------|--------------|--------------|
|      A1     |   19673   |   19709   |  19745 |     19703    |     19670    |
|      A2     |   40477   |   40372   |  40528 |     40482    |     40222    |
|      A3     |   42193   |   41976   |  42198 |     42135    |     41867    |
|      A4     |   27488   |   27237   |  27493 |     27488    |     27269    |
|      B1     |   28604   |   28454   |  28604 |     28604    |     28403    |
|      B2     |   18816   |   18734   |  18836 |     18730    |     18671    |
|      B3     |   14147   |   14141   |  14147 |     14092    |     14118    |
|      B4     |   44376   |   44272   |  44424 |     44114    |     44016    |
|      C1     |   17947   |   17796   |  17969 |     17938    |     17709    |
|      C2     |   27871   |   27859   |  27894 |     27853    |     27871    |
|      C3     |   19555   |   19503   |  19555 |     19408    |     19503    |
|      C4     |   28554   |   28474   |  28556 |     28521    |     28550    |
|      D1     |    2333   |    2332   |   2333 |      2333    |      2333    |
|      D2     |    5630   |    5603   |   5630 |      5621    |      5620    |
|      D3     |   29101   |   28975   |  29103 |     29099    |     28899    |
|      D4     |   45497   |   45440   |  45498 |     45154    |     45498    |

## Tratamiento de los conjuntos de datos

### Generación de un *DataFrame* con los archivos CSV concatenados
Se crea un *DataFrame* consolidado, nombrado **df_combined**, a partir de la concatenación vertical de los archivos CSV, en el supuesto de que todos éstos comparten las mismas columnas. Este último supuesto se confirma en la segunda parte del análisis exploratorio.

---

### Conversión de columnas "Epoch" y "Timestamp" de los CSV originales o *DataFrame* consolidado
Estas columnas, que son almacenadas por Pandas como de tipo *int* y *object*, respectivamente, deben ser convertidas a *datetime* para su optimización y posterior tratamiento.

Luego de dicha conversión, se obtuvo que:
* Ambas columnas resultan en dos timestamps con una diferencia de 3 horas entre el valor de una y la otra.
* "Epoch" está 3 horas por sobre "Timestamp".
* Por consiguiente, la columna "Timestamp" está en UTC-03, que fue la zona horaria de Chile al momento de tomar las mediciones.

---

### Verificación de datos nulos o filas duplicadas
Una revisión del CSV consolidado dio como resultado:
* Cantidad de valores *NaN*: 0
* Cantidad de filas duplicadas: 0

A pesar de que ya se realizó una primera búsqueda de valores nulos en el *DataFrame* consolidado, se efectuó una segunda revisión, archivo por archivo y a través de código, para validar el resultado anterior.

De las 400 columnas que contiene la totalidad de archivos (16 dispositivo x 5 archivos x 5 columnas), ninguna posee valores *NaN* (nulos).

---

### Creación de una tabla pivote para fusionar filas con un mismo "Timestamp" y "Node"
A partir del *DataFrame* consolidado, se genera una tabla pivote en que se fusionan los registros que poseen un mismo "TimeStamp" y "Node",  para así desplegar los resultados por dispositivo y hora de registro de medición de manera transversal, esto es, entre los distintos archivos CSV.

La tabla pivote posee las siguientes columnas:
* "Timestamp" (índice del *df*)
* "Node" (índice del *df*)
* "Slot":
    * "Humedad 1"
    * "Humedad 2"
    * "Temperatura 1"
    * "Temperatura 2"
    * "pH"

---

### Descargar CSV con los conjuntos de datos concatenados
Finalmente, se habilita la opción de descargar el *DataFrame* consolidado en formato CSV, con las columnas "Epoch" y "Timestamp" no convertidas a *datetime*, para su exportación y uso posterior.